Кароче парсер честно взял у этого мужичка [github](https://github.com/yutkin/Lenta.Ru-News-Dataset), так как самому было лень это парсить, а пример от Лены мне не особо понравился (я не особо понял номера type, bloc, domain, а доку не нашел, так что бросил это дело). Вооот парсер подправил, тк теги на страницах успели уже поменять + добавил, чтоб он пробегался по страницам (10 по дефолту)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('lenta-ru-news.csv')

In [3]:
df.head()

,url,title,text,topic,tags
0,https://lenta.ru/news/2024/01/01/rastem/,В России увеличились пенсии,С 1 января 2024 года в России выросли страховы...,Экономика,Социальная сфера
1,https://lenta.ru/news/2024/01/01/age/,В России подняли призывной возраст до 30 лет,С 1 января 2024 года начал действовать закон о...,Россия,Политика
2,https://lenta.ru/news/2024/01/01/to/,В России подорожал техосмотр автомобилей,С 1 января 2024 года в России подорожала проце...,Россия,Политика
3,https://lenta.ru/news/2024/01/01/matkapital/,В России изменились условия выдачи материнског...,С 1 января 2024 года материнский капитал будут...,Россия,Политика
4,https://lenta.ru/news/2024/01/01/obnulyaem/,В России обнулили пошлины на ввоз яиц из-за ру...,Правительство России обнулило пошлины на ввоз ...,Экономика,Рынки


In [4]:
df.topic.value_counts()

topic
Россия               20301
Мир                  19244
Бывший СССР          13302
Экономика             9322
Силовые структуры     8708
Спорт                 4366
Интернет и СМИ        4306
Наука и техника       4008
Ценности              2917
Путешествия           2914
Из жизни              2904
Культура              2423
Забота о себе         2307
Среда обитания        1156
69-я параллель         168
Моя страна             142
Бизнес                   2
Name: count, dtype: int64

In [5]:
df.tags.value_counts()

tags
Политика           24385
Украина            10574
Происшествия        7833
Общество            7036
Следствие и суд     5900
                   ...  
Культура               3
В России               3
69-я параллель         2
Туризм                 1
Мировой бизнес         1
Name: count, Length: 88, dtype: int64

Видно после парса очень много тегов и топиков, которые по заданию нам не нужны, так что попробуем вычистить и оставить только те, которые нам нужны

* 'Общество/Россия' : 0
* 'Экономика' : 1
* 'Силовые структуры' : 2
* 'Бывший СССР' : 3
* 'Спорт' : 4
* 'Забота о себе' : 5
* 'Строительство' : 6
* 'Туризм/Путешествия' : 7
* 'Наука и техника' : 8

In [6]:
df0 = df[(df.topic == 'Россия') & (df.tags == 'Общество')]
df1 = df[(df.topic == 'Экономика') & (df.tags != 'Недвижимость')]
df2 = df[df.topic == 'Силовые структуры']
df3 = df[df.topic == 'Бывший СССР']
df4 = df[df.topic == 'Спорт']
df5 = df[df.topic == 'Забота о себе']
df6 = df[(df.tags == 'Недвижимость')]
df7 = df[df.topic == 'Путешествия']
df8 = df[(df.topic == 'Наука и техника')]

In [9]:
df0.loc[:,'target'] = 0
df1.loc[:,'target'] = 1
df2.loc[:,'target'] = 2
df3.loc[:,'target'] = 3
df4.loc[:,'target'] = 4
df5.loc[:,'target'] = 5
df6.loc[:,'target'] = 6
df7.loc[:,'target'] = 7
df8.loc[:,'target'] = 8

In [10]:
new_df = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8]).reset_index(drop=True)

In [11]:
new_df['target'].value_counts()

target
3    13302
2     8708
1     8469
0     6571
4     4366
8     4008
7     2914
5     2307
6      853
Name: count, dtype: int64

Видно получился дизбаланс классов, и у нас мало очень новостей по строительству. Можно попытататься сделать апсемплинг данных малых по численности классов, можно попытаться напарсить побольше новостей под этот класс, но попробую в начале на голую обучиться без ничего (лень парсер еще переделывать, чтоб он определенную тему подтягивал). И если прям качество будет плохим, то докину. 

Меня еще смущает, что способом, которым я парсил, не выходит текст такой же как на тесте, так что тест придется поправлять скорее всего (убирать по типу "Фото: ...")

Примеры текстов:

Россия/общество

In [51]:
df0.sample(1)['text'].iloc[0]

'В Приморском районе Санкт-Петербурга младенец захлебнулся рвотными массами после ночного кормления. Об этом сообщила «Фонтанка». По сведениям издания, накануне 32-летняя мать вызвала бригаду медиков в квартиру на Комендантском проспекте. Она рассказала, что покормила восьмимесячную дочь и уложила ее спать. По словам россиянки, у ребенка начались колики и рвота. Прибывшим врачам реанимировать малышку не удалось. По факту произошедшего была организована проверка. Ранее стало известно, что в Подмосковье медики нашли у годовалого ребенка в правом бронхе стик от системы нагревания табака. Его родители не стали из-за этого предпринимать каких-либо действий. Медики из областного центра охраны материнства и детства смогли удалить устройство через дыхательные пути, не делая дополнительных разрезов.'

Ужастики....

Экономика

In [52]:
df1.sample(1)['text'].iloc[0]

'Наблюдающаяся на российском валютном рынке волатильность является следствием новых антироссийских санкций и носит эмоциональный характер, считает аналитик «Цифра брокер» Наталия Пырьева. Ее слова приводит «Коммерсантъ». Текущую динамику курса доллара специалист связала с отсутствием спроса на любую валюту, связанную с «непониманием того, как будут производиться дальнейшие расчеты». По ее словам, как только нынешняя неопределенность на рынках завершится, американская валюта начнет расти, а ее курс будет определяться показателями внешней торговли. Исходя из показателей платежного баланса РФ, Пырьева полагает, что речь пойдет о возвращении стоимости доллара к диапазону от 87 до 93-94 рублей. Дальнейшее «не вполне оправданное» падение курса юаня, снижающегося вместе с переставшими торговаться на Мосбирже долларом и евро, может потребовать вмешательства Банка России, считает начальник отдела экономического и отраслевого анализа Промсвязьбанка (ПСБ) Евгений Локтюхов. Он отметил, что слабый 

Силовые структуры

In [53]:
df2.sample(1)['text'].iloc[0]

'Задержание IT-специалиста Фируза Дадобоева за попытку передачи секретных данных Центральному разведывательному управлению (ЦРУ) попало на видео. Соответствующую видеозапись опубликовал ТАСС со ссылкой на Центр общественных связей (ЦОС) ФСБ России. На кадрах видно, как силовики выходят из автомобиля, подбегают к идущему по улице мужчине и кладут его на землю. Они надевают задержанному наручники, поднимают, накрывают его голову мешком и обыскивают. Затем показано, как силовики осматривают его рюкзак. Внутри они находят документы и иностранную валюту. После этого фигуранта сажают в машину. Во второй части видео показано, как он находится в суде, где судья зачитывает ему приговор. Отмечается, что Московский городской суд приговорил Дадобоева к 13,5 года лишения свободы по статье о госизмене. Он будет отбывать наказание в колонии строгого режима и выплатит штраф в размере 300 тысяч рублей.'

Бывший СССР

In [63]:
df3.sample(1)['text'].iloc[0]

'Премьер-министр Грузии Ираклий Кобахидзе не исключил, что в случае аннулирования мандатов депутатам оппозиционной партии «Единое национальное движение» (ЕНД), парламент после выборов 26 октября может стать однопартийным. Об этом пишет газета «Взгляд». Кобахидзе отметил, что Грузия в этом плане может повторить опыт Молдавии. «Парламент Молдовы практически однопартийный, однако все его приветствуют. То же самое произойдет и в Грузии», — сказал глава правительства, добавив, что «стандарты должны быть едиными для всех стран-кандидатов» в Европейский союз — Грузии, Молдавии, Украины. Ранее Кобахидзе заявил, что власти Грузии планируют запретить партию бывшего президента страны Михаила Саакашвили «Единое национальное движение» и другие оппозиционные объединения. Это может произойти после победы правящей партии на предстоящих выборах. «Это одна коллективная криминальная политическая сила, которая обязательно должна быть осуждена», — пояснил грузинский премьер. В частности, Кобахидзе обвинил 

Чот бывший СССР не особо понимаю, это новости СНГ?

Спорт

In [64]:
df4.sample(1)['text'].iloc[0]

'Трехкратная олимпийская чемпионка по фигурному катанию и депутат Госдумы Ирина Роднина ответила канадской фигуристке Меган Дюамель, выступившей за ужесточение правил для сменивших гражданство россиян. Ее слова приводит «ВсеПроСпорт». Роднина отметила, что в Канаде тренируется множество европейских фигуристов. «Дюамель это прекрасно знает, поскольку сама занималась в таких центрах. А если у нее чешется язык в отношении российских фигуристов, это ее проблемы. Авторитетно ли ее мнение? Конечно, нет», — заявила она. Бывшая фигуристка также усомнилась в умственных способностях Дюамель. «Видимо, она не особо училась, а только тренировалась», — добавила Роднина. Ранее Дюамель призвала ужесточить правила в отношении сменивших гражданство российских фигуристов. «Считаю, что у вас должно быть право покинуть Россию и сменить страну, но фигуристы должны тренироваться в той стране, которую они решили представлять», — заявила она.'

Забота о себе

In [65]:
df5.sample(1)['text'].iloc[0]

'Скумбрия имеет сильное противовоспалительное действие, разжижает кровь и стабилизирует артериальное давление. О неожиданной пользе рыбы рассказала эндокринолог Екатерина Янг в Telegram. По словам Янг, скумбрия является рекордсменом по содержанию жирных кислот омега-3 и витамина D. Эти вещества усиливают устойчивость организма к инфекциям, а также помогают бороться с лишним весом. Кроме того, Янг назвала скумбрию полезной для уменьшения воспаления и здоровья сосудов. «Повышает количество полезного холестерина. Улучшает эластичность сосудов, текучесть крови, нормализует артериальное давление», — уточнила специалистка. Еще один продукт, который эффективно борется с воспалением, — это чеснок, считает Янг. Она указала, что этот продукт снижает артериальное давление, способствует разжижению крови и уменьшает уровень плохого холестерина. Кроме того, чеснок помогает укрепить иммунитет и поддерживает работу кишечника, заключила эндокринолог. Ранее врач-диетолог Джулия Зумпано рассказала, что п

Строительство

In [66]:
df6.sample(1)['text'].iloc[0]

'Спрос на покупку недвижимости за рубежом среди россиян снизился до 40 процентов. О том, что граждане практически  отвернулись от иностранного жилья, сообщают «Известия». Эксперты связывают такую динамику с сокращением иммиграционного потока, нестабильностью курса рубля, закрытием паспортных программ для получения россиянами иностранного гражданства и сложностями со сделками на фоне санкций. По данным Intermark Global, за первые полгода 2024-го российский спрос на объекты в других странах упал на 19 процентов относительно аналогичного периода 2023 года. Согласно подсчетам Urban, покупательская активность сократилась еще сильнее — на 30-40 процентов в годовом выражении. Отмечается, что при этом россияне стали более взвешенно планировать приобретение такой недвижимости и дольше принимать решения. Ранее спикер Госдумы Вячеслав Володин предложил запретить депутатам владеть недвижимостью за рубежом.'

Туризм/Путешествия

In [73]:
df7.sample(1)['text'].iloc[0]

'Россия обсуждает введение безвизового режима с двумя африканскими странами, речь идет о Кении и Чаде. Об этих планах сообщили «Известия». Отмечается, что эти направления набирают популярность среди российских туристов. Так, число поездок в Кению увеличилось на 40 процентов. «Отмена виз облегчает путешествия и делает поездки более доступными, может положительно сказаться на экономике Кении, в частности, на гостиничном бизнесе и других связанных секторах. Кения всегда была популярным направлением среди туристов из различных стран, включая Россию», — отметил глава комиссии по работе с африканскими диаспорами российско-африканского клуба Луис Гоуенд. При этом в России вызывает беспокойство вопрос безопасности. Эта проблема обострилась на фоне протестов в Кении. Ранее министр экономического развития Максим Решетников заявил, что Россия надеется на отмену визы с шестью странами. Уже ведутся переговоры с Саудовской Аравией, Бахрейном, Кувейтом, Оманом, Малайзией, Мьянмой.'

Наука и техника

In [75]:
df8.sample(1)['text'].iloc[0]

'Конструкторское бюро (КБ) «Талламхо» (Грозный) создало оптическую систему выявления снайперов и диверсантов «Ронин», которая для испытаний будет направлена в зону специальной военной операции (СВО). Об этом сообщает ТАСС. В КБ уточнили, что изделие выполняет три функции: разведку (вычисление географических координат), охрана (выявление оптики противника) и наведение (охрана с привязкой к оружейной системе). Окончательно работы над системой планируется завершить в феврале. «Изделие уже испытывалось, но не применялось в боевых условиях. В настоящее время проходит заключительный этап разработки, рассчитываем, что в феврале система будет полностью готова. В дальнейшем планируется провести испытания аппарата в зоне спецоперации», — добавили в КБ. Внешне «Ронин» выглядит как десятидюймовый планшет, с обратной стороны которого прикреплены блок питания и плата. Ранее специалисты грозненского КБ разработали систему акустической разведки БУХ1а («Сова»), которую позиционируют как бюджетный анало

Сееейв

In [13]:
new_df.to_csv('train.csv')